In [1]:
# 第一部分為前置作業設定
from bs4 import BeautifulSoup
import requests
import time
import json
import csv
import re
import pandas as pd

# 宣告list並將使用到的list清空以免重複執行資料不正確
link = []
temp_url = []
word_list = []
url = []
init = []
key = []
info = []
update = []
synonym = {}

# 讀取使用者設定
with open('./config.txt', 'r', encoding='utf-8') as f:
    conf = f.read().split("\n")
for i in range(len(conf)):
    init.append(conf[i].split('：')[1])
# print(init)

with open('./dict.txt', 'r', encoding='utf-8') as f:
    skill_dict = f.read().split('\n')

# 同義字的字典
# 功能 -> 遇到'人工智慧'時，想要用key,value的特性轉成'AI'
with open('./synonym.txt', 'r', encoding='utf-8') as f:
    syn = f.read().split('\n')
synonym_data = {}
for i in range(len(syn)):
    synonym_data.setdefault((syn[i].split(':')[0]), syn[i].split(':')[1])
    
# 將ro(是否只找全職工作)、keyword(關鍵字)、page(頁數)帶入網址，並存在新的list內
for page in range(1, int(init[1])+1):
    url.append('https://www.104.com.tw/jobs/search/?ro={}&keyword={}&order=1&page={}&jobsource=2018indexpoc&'.format(init[2], init[0],
                                                                                                              page))

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36',
           }

sync = ['Python', 'java', 'SQL', 'AI', 'data mining', 'Linux', 'ML', 'DL']
columns = ['area','company', 'job_title', 'content', 'experience', 'education', 'language ability', 'pay', 'welfare', 'update','URL',
          'Python', 'java', 'SQL', 'AI', 'data mining', 'Linux', 'ML', 'DL']
df = pd.DataFrame(columns=columns)

# 工作內容、薪資、福利透過工作連結取得
def get_job_info(sub_soup):
    try:                # 擷取工作地點
        area = sub_soup.select('title')[0].text.split('｜')[2].split('－')[0]
    except:
        area = 'Area Error'
    try:                # 擷取公司名稱
        company = sub_soup.select('title')[0].text.split('｜')[1]
    except:
        company = 'Company Error'
    try:                # 擷取職稱
        title = sub_soup.select('title')[0].text.split('｜')[0]
    except:
        title = 'Title Error'
    try:                # 擷取工作經歷
        experience = sub_soup.find_all("table", class_="column2 condition")[0].text.split('工作經歷：\n')[1].split('\n\n')[0]
    except:  
        Experience = 'Experience Error'
    try:                # 擷取學歷
        education = sub_soup.find_all("table", class_="column2 condition")[0].text.split('學歷要求：\n')[1].split('\n\n')[0]
    except:
        education = 'Education Error'
    try:                # 擷取語言能力
        language = sub_soup.find_all("table", class_="column2 condition")[0].text.split('語文條件：\n\n')[1].split('\n\n')[0]
    except:
        language = '不拘'
    try:                # 擷取薪資
        pay = sub_soup.select('div.content')[2].text.split('工作待遇：\n\n')[1].split('\n')[0]  
    except:
        pay = 'Pay Error'
    try:                # 擷取福利
        welfare = sub_soup.select('div.content')[3].text 
    except:
        welfare = 'welfare Error'
    try:                # 擷取工作內容
        content = sub_soup.select('div.content')[1].text   
    except:
        content = 'Content Error'
    try:                #擷取擅長工具，不一定有東西
        skill = sub_soup.find_all("table", class_="column2 condition")[0].text.split('擅長工具：\n')[1].split('\n\n')[0]
    except:
        skill = ' '
    try:                # 擷取其他條件，不一定有東西
        addition = sub_soup.find_all("table", class_="column2 condition")[0].text.split('其他條件：\n\n\n')[1]
    except:
        addition = ' '

    # return只會傳str，需要將上面取得之內容放進list內整個回傳， 否則會只回傳第一個字元  
    tmp_list = [area, company, title, experience, education, re.sub(' ', '', language), re.sub(' ', '', pay), re.sub('[\n\t\r]', '', welfare),  content, skill, re.sub('[\n\t\r]', '', addition)]
    return tmp_list[0], tmp_list[1], tmp_list[2], tmp_list[3], tmp_list[4], tmp_list[5], tmp_list[6], tmp_list[7], tmp_list[8], tmp_list[9], tmp_list[10]


# 將同義詞的key值全轉成大寫後儲存新的字典以用來和同義詞做比對，value不變
new_dict = {}
for i, j in synonym_data.items():
    new_dict[i.upper()] = j
# print(new_dict)

In [2]:
# ========================透過session get訪問並透過html取得所需要的資訊 =========================
ss = requests.session()
for now_page in range(0, len(url)):
    print('開始第{}頁爬蟲'.format(now_page+1))
    res = ss.get(url=url[now_page], headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    #帶關鍵字送出request後會進到工作列表，此時可以透過article的class屬性擷取更新日期、進入工作頁面的連結
    soup2 = soup.find('div',{'id':'js-job-content'}).findAll('article',
                                                              {'class':'b-block--top-bord job-list-item b-clearfix js-job-item'})
    for job in soup2:
        update_date = job.find('span',{'class':'b-tit__date'}).text
        update.append(re.sub('\r|\n| ','',update_date))
        temp_url.append(job.find('a',{'class':'js-job-link'})['href'].replace("//", "https://").split("?jobsource=")[0]) # 連結 

    
    # =================================透過工作連結進入子頁面爬蟲============================================
    # 前面取到的網址改用使用手機板網址進行訪問(www改成m即可)
    # 將轉換過的html(sub_soup_main)帶入get_job_info的function中    
    print('此頁面共找到{}筆資料'.format(len(temp_url)))
    for u in range(0, len(temp_url)):
        if (u%5 == 0):
            print('休息5秒')
            time.sleep(5)
        print('進入職缺頁面{}取得更多資料'.format(u+1))
        time.sleep(1)
        sub_res = ss.get(url=temp_url[u].replace("www", "m"), headers=headers)
        sub_soup_main = BeautifulSoup(sub_res.text, 'html.parser')
        area_tmp, company_tmp, title_tmp, exp_tmp, edu_tmp, language_tmp, pay_tmp, welfare_tmp, content_tmp, skill_tmp, addition_tmp = get_job_info(sub_soup_main)
        
        # "工作內容"直接存進輸出的列表中
        # 另外準備一個key_word_content將"工作內容"加上"擅長工具"、"其他條件"作為要篩選關鍵字的字串集合
        word_clean = (re.sub(r'[-:_0-9、【】：)(，.&+]', '', content_tmp)).replace('\n', '').replace('\r', ' ')
        key_word_content = word_clean+skill_tmp+addition_tmp
        
        link.append(temp_url[u])
        info.append([area_tmp, company_tmp, title_tmp, word_clean, exp_tmp, edu_tmp, language_tmp, pay_tmp, welfare_tmp, update[u], temp_url[u]])
        
        
        # =================================關鍵字比對============================================
        
        # tmp用來暫存這個工作的找到的所有關鍵字，每次都要先清空
        tmp = []
        # 第一個判斷式將關鍵字的字串與工作內容比對，第二個判斷式再比對該字串是否出現在同義詞中，若有則將value存進tmp中，
        for b in range(len(skill_dict)):
            if skill_dict[b].upper() in key_word_content.upper():
#                 print(skill_dict[b])
                if skill_dict[b].upper() in new_dict.keys():
                    tmp.append(new_dict[skill_dict[b].upper()])
                else:
                    tmp.append(skill_dict[b])
        # set作為資料結構，在設計上保證內部元素都是唯一的，會自動過濾重複資料，只要先轉換成set再轉回list就可以把重複的刪除           
        word_list.append(list(set(tmp)))  
    temp_url = []
# print('info=', info)

 # =================================儲存關鍵字比對結果並存成list============================================
# 預設所有關鍵字為0，並加入到二維list中
for t in range(len(link)):
    key_tmp = []  # 每次都將list清空否則會不斷疊加
    for p in range(len(sync)):
        key_tmp.insert(p, 0)
    key.append(key_tmp)

    # 若符合list中的字串，則該位置將設成1，最後會得到一個關鍵字的二維表
    for k in word_list[t]:
        if k in sync:
            key[t][sync.index(k)]=1
# print(key)


開始第1頁爬蟲
此頁面共找到20筆資料
休息5秒
進入職缺頁面1取得更多資料
進入職缺頁面2取得更多資料
進入職缺頁面3取得更多資料
進入職缺頁面4取得更多資料
進入職缺頁面5取得更多資料
休息5秒
進入職缺頁面6取得更多資料
進入職缺頁面7取得更多資料
進入職缺頁面8取得更多資料
進入職缺頁面9取得更多資料
進入職缺頁面10取得更多資料
休息5秒
進入職缺頁面11取得更多資料
進入職缺頁面12取得更多資料
進入職缺頁面13取得更多資料
進入職缺頁面14取得更多資料
進入職缺頁面15取得更多資料
休息5秒
進入職缺頁面16取得更多資料
進入職缺頁面17取得更多資料
進入職缺頁面18取得更多資料
進入職缺頁面19取得更多資料
進入職缺頁面20取得更多資料
開始第2頁爬蟲
此頁面共找到20筆資料
休息5秒
進入職缺頁面1取得更多資料
進入職缺頁面2取得更多資料
進入職缺頁面3取得更多資料
進入職缺頁面4取得更多資料
進入職缺頁面5取得更多資料
休息5秒
進入職缺頁面6取得更多資料
進入職缺頁面7取得更多資料
進入職缺頁面8取得更多資料
進入職缺頁面9取得更多資料
進入職缺頁面10取得更多資料
休息5秒
進入職缺頁面11取得更多資料
進入職缺頁面12取得更多資料
進入職缺頁面13取得更多資料
進入職缺頁面14取得更多資料
進入職缺頁面15取得更多資料
休息5秒
進入職缺頁面16取得更多資料
進入職缺頁面17取得更多資料
進入職缺頁面18取得更多資料
進入職缺頁面19取得更多資料
進入職缺頁面20取得更多資料
開始第3頁爬蟲
此頁面共找到20筆資料
休息5秒
進入職缺頁面1取得更多資料
進入職缺頁面2取得更多資料
進入職缺頁面3取得更多資料
進入職缺頁面4取得更多資料
進入職缺頁面5取得更多資料
休息5秒
進入職缺頁面6取得更多資料
進入職缺頁面7取得更多資料
進入職缺頁面8取得更多資料
進入職缺頁面9取得更多資料
進入職缺頁面10取得更多資料
休息5秒
進入職缺頁面11取得更多資料
進入職缺頁面12取得更多資料
進入職缺頁面13取得更多資料
進入職缺頁面14取得更多資料
進入職缺頁面15取得更多資料
休息5秒
進入職缺頁面16取得更多資料
進入職缺頁面17取得更多資料
進入職缺頁面18取得更多資料
進入職缺頁面19取得更多資料
進入職缺頁面20取得更多資料
開始第4頁爬蟲

In [3]:
# 寫入dataframe
for u in range(0,len(link)):
     df.loc[u] = [info[u][0], info[u][1], info[u][2], info[u][3], info[u][4], info[u][5],
                  info[u][6], info[u][7], info[u][8], info[u][9], info[u][10],
                  key[u][0], key[u][1], key[u][2], key[u][3],
                  key[u][4], key[u][5], key[u][6], key[u][7]
                  ]
# 寫入CSV檔        
df.to_excel("./104ETL_數據工程師.xlsx", encoding='utf-8',index=False)
df

,area,company,job_title,content,experience,education,language ability,pay,welfare,update,URL,Python,java,SQL,AI,data mining,Linux,ML,DL
0,台北市大安區,艾斯移動股份有限公司,大數據工程師,▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬ 艾 斯 移 動 ▬▬▬▬▬▬▬▬▬▬...,不拘,大學以上,英文--聽/略懂、說/略懂、讀/略懂、寫/略懂,"月薪60,000~85,000元薪資行情",▍ 上班時間 09:00~18:00 (...,8/23,https://www.104.com.tw/job/6nmju,1,1,1,0,1,0,0,0
1,桃園市桃園區,安智聯科技有限公司,大數據工程師,工作說明 搭建分散式系統 維運分散式系統及其上的job 與PM溝通產...,2年以上,大學、碩士,英文--聽/中等、說/略懂、讀/中等、寫/略懂,"月薪55,000元以上薪資行情",法定項目週休二日 其他福利慶生會 員工紅利 年終獎金 三節獎金/禮品 ...,8/23,https://www.104.com.tw/job/7bbe1,1,1,1,0,0,1,0,0
2,台中市北區,優聘資訊科技有限公司,[TCXY08]大數據工程師_Big Data Engineer,代企業徵才錄取後為該企業正職員工工作職責 以集團內部之業務資料進行資...,1年以上,大學以上,英文--聽/略懂、說/略懂、讀/中等、寫/中等,"月薪38,000~80,000元薪資行情",出差外派：無需出差外派上班時段：日班，8:30-17:30休假制度：週休二日管理責任：不需負...,8/20,https://www.104.com.tw/job/72x55,1,1,0,1,1,0,0,0
3,台北市南港區,博客來數位科技股份有限公司,大數據工程師,執行系統維護與專案系統的開發系統測試撰寫程式說明文件及測試案例文件進...,3年以上,大學以上,英文--聽/略懂、說/略懂、讀/略懂、寫/略懂,待遇面議(經常性薪資達4萬元或以上),1.彈性上、下班制2.福委會組織3.四節禮金...,8/19,https://www.104.com.tw/job/74dad,1,1,1,0,1,0,0,0
4,桃園市楊梅區,達康國際有限公司,大數據工程師,開發大數據系統資料處理平台串流資料平行化處理程序開發熟悉分散式運算平...,不拘,大學以上,不拘,待遇面議(經常性薪資達4萬元或以上),法定項目勞保、健保、加班費、勞退提撥金福利制...,8/19,https://www.104.com.tw/job/77l19,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,台北市信義區,國泰世華商業銀行股份有限公司_人力資源部,"【數據】資料科學工程師 (數數發中心,DDT)",職務說明 What will you do數據介接清洗處理與數據模型...,不拘,大學以上,英文--聽/中等、說/中等、讀/中等、寫/中等,待遇面議(經常性薪資達4萬元或以上),【專屬於你的幸福】1. 三節獎金(春節、中秋...,8/19,https://www.104.com.tw/job/6v7pz,1,0,0,1,1,1,1,1
196,桃園市中壢區,迅得機械股份有限公司,機器學習數據應用開發工程師(110年研發替代役),智慧設備邊緣運算控制與數據整合分析應用開發。 智慧工廠整廠設備自動...,不拘,大學以上,英文--聽/中等、說/中等、讀/中等、寫/中等,待遇面議(經常性薪資達4萬元或以上),◆ 分紅/配股1.員工紅利2.員工認股3.員...,8/20,https://www.104.com.tw/job/747kz,1,0,1,1,1,0,1,0
197,高雄市左營區,公勝保險經紀人股份有限公司,資訊部 數據分析工程師(總公司),建立推廣分析架構並提出建議以透過數據分析提升對業務夥伴及客戶之洞察並...,2年以上,大學以上,不拘,"月薪30,000~50,000元薪資行情",法定項目哺乳室 週休二日 家庭照顧假 勞保 健保 陪產假 產假 特別休假 育嬰留停 女性生理...,8/25,https://www.104.com.tw/job/7azj8,1,0,0,0,1,0,0,0
198,台北市信義區,國泰金控_國泰金融控股股份有限公司,"資料倉儲工程師 Data Engineer -集團徵才-數據科技(數數發中心, DDT)",[團隊介紹與文化特色] 團隊特色我們是數據分析團隊專注在數據創新商業...,不拘,大學以上,英文--聽/精通、說/精通、讀/精通、寫/精通,"月薪30,000~90,000元薪資行情",法定項目週休二日 其他福利不扣薪病假 家庭日 員工貸款 結婚禮金 生育津貼 社團補助 員工進...,8/20,https://www.104.com.tw/job/6s9b8,1,0,1,0,1,0,0,0
